***GENERATED CODE FOR bankdepositclfmanualml PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'job', 'transformation_label': 'String Indexer'}], 'feature': 'job', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'admin.', 'max': 'unknown', 'missing': '0', 'distinct': '12'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'job'}, {'feature_label': 'job', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('job')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'marital', 'transformation_label': 'String Indexer'}], 'feature': 'marital', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'divorced', 'max': 'single', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'marital'}, {'feature_label': 'marital', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('marital')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'education', 'transformation_label': 'String Indexer'}], 'feature': 'education', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'primary', 'max': 'unknown', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'education'}, {'feature_label': 'education', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('education')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'default', 'transformation_label': 'String Indexer'}], 'feature': 'default', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'default'}, {'feature_label': 'default', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('default')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'housing', 'transformation_label': 'String Indexer'}], 'feature': 'housing', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'housing'}, {'feature_label': 'housing', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('housing')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'loan', 'transformation_label': 'String Indexer'}], 'feature': 'loan', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'loan'}, {'feature_label': 'loan', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('loan')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'contact', 'transformation_label': 'String Indexer'}], 'feature': 'contact', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'cellular', 'max': 'unknown', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'contact'}, {'feature_label': 'contact', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('contact')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'month', 'transformation_label': 'String Indexer'}], 'feature': 'month', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'apr', 'max': 'sep', 'missing': '0', 'distinct': '11'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'month'}, {'feature_label': 'month', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('month')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'poutcome', 'transformation_label': 'String Indexer'}], 'feature': 'poutcome', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'failure', 'max': 'unknown', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'poutcome'}, {'feature_label': 'poutcome', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('poutcome')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'deposit', 'transformation_label': 'String Indexer'}], 'feature': 'deposit', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'deposit'}, {'feature_label': 'deposit', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('deposit')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
import pyspark
from sklearn.metrics import accuracy_score
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
import xgboost as xgb
import numpy as np
import lightgbm as lgb
import optuna
from sklearn import preprocessing
lbl = preprocessing.LabelEncoder()


class GradientBoostingClassifierOptunaDriverClass:
    def set_tag(trial, df_new, features, label):
        def objective(trial):
            cat_col = df_new.select_dtypes(
                exclude=['int', 'float']).columns.values
            for c in cat_col:
                df_new[c] = df_new[c].astype('category')
                df_new[c] = lbl.fit_transform(df_new[c].astype(str))
            XRaw = df_new[features].values
            yRaw = df_new[label].values
            X_train, X_test, y_train, y_test = train_test_split(
                XRaw, yRaw, random_state=42)
            max_depth = trial.suggest_int("max_depth", 2, 32)
            n_est = trial.suggest_int('n_estimators', 1, 50)
            max_features = trial.suggest_categorical(
                'max_features', ['auto', 'sqrt', 'log2'])
            max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 2, 10)
            classifier_obj = GradientBoostingClassifier(max_depth=max_depth, n_estimators=n_est,
                                                        max_features=max_features, max_leaf_nodes=max_leaf_nodes)

            classifier_obj.fit(X_train, y_train)
            preds = classifier_obj.predict(X_test)
            acc = accuracy_score(np.round(preds), y_test).mean()
            return acc
        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=10, timeout=600)
        best_parameters = study.best_trial.params

        return best_parameters


class RandomForestClassifierOptunaDriverClass:
    def set_tag(trial, df_new, features, label):
        def objective(trial):
            cat_col = df_new.select_dtypes(
                exclude=['int', 'float']).columns.values
            for c in cat_col:
                df_new[c] = df_new[c].astype('category')
                df_new[c] = lbl.fit_transform(df_new[c].astype(str))
            XRaw = df_new[features].values
            yRaw = df_new[label].values
            X_train, X_test, y_train, y_test = train_test_split(
                XRaw, yRaw, random_state=42)

            rf_max_depth = int(trial.suggest_int("max_depth", 2, 15))
            n_est = trial.suggest_int('n_estimators', 1, 50)
            max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 2, 10)
            max_features = trial.suggest_categorical(
                'max_features', ['auto', 'sqrt', 'log2'])
            classifier_obj = RandomForestClassifier(
                max_depth=rf_max_depth, n_estimators=n_est, max_features=max_features, max_leaf_nodes=max_leaf_nodes)
            classifier_obj.fit(X_train, y_train)
            preds = classifier_obj.predict(X_test)
            acc = accuracy_score(np.round(preds), y_test).mean()
            return acc
        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=10, timeout=600)
        best_parameters = study.best_trial.params

        return best_parameters


class LightGBMClassifierOptunaDriverClass:
    def set_tag(trial, df_new, features, label):
        def objective(trial):
            cat_col = df_new.select_dtypes(
                exclude=['int', 'float']).columns.values
            for c in cat_col:
                df_new[c] = df_new[c].astype('category')
                df_new[c] = lbl.fit_transform(df_new[c].astype(str))
            XRaw = df_new[features].values
            yRaw = df_new[label].values
            X_train, X_test, y_train, y_test = train_test_split(
                XRaw, yRaw, random_state=42)
            param = {
                'task': 'train',
                'boosting_type': 'gbdt',
                'objective': 'binary',
                'metric': 'binary_logloss',
                "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 10.0),
                "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 10.0),
                "num_leaves": trial.suggest_int("num_leaves", 2, 50),
                "feature_fraction": trial.suggest_uniform("feature_fraction", 0.4, 1.0),
                "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.4, 1.0),
                "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
                "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
            }
            y_train_copy = [item for sublist in y_train for item in sublist]
            if len(list(set(tuple(y_train_copy)))) > 2:
                param['num_class'] = len(list(set(y_train_copy)))
                param['objective'] = 'multiclass'
                param['metric'] = 'multi_logloss'

            gbm = lgb.LGBMClassifier(**param)
            gbm.fit(X_train, y_train)

            preds = gbm.predict(X_test)
            acc = accuracy_score(np.round(preds), y_test).mean()

            return acc

        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=10, timeout=600)
        best_parameters = study.best_trial.params

        return best_parameters


class xgboostClassifierOptunaDriverClass:
    def set_tag(trial, df_new, features, label):
        def objective(trial):
            cat_col = df_new.select_dtypes(
                exclude=['int', 'float']).columns.values
            for c in cat_col:
                df_new[c] = df_new[c].astype('category')
                df_new[c] = lbl.fit_transform(df_new[c].astype(str))
            XRaw = df_new[features].values
            yRaw = df_new[label].values
            X_train, X_test, y_train, y_test = train_test_split(
                XRaw, yRaw, random_state=42)

            n_estimators = trial.suggest_int('n_estimators', 1, 50)
            max_depth = trial.suggest_int('max_depth', 1, 10)
            min_child_weight = trial.suggest_int('min_child_weight', 1, 10)
            scale_pos_weight = trial.suggest_int('scale_pos_weight', 1, 50)
            subsample = trial.suggest_discrete_uniform(
                'subsample', 0.5, 0.9, 0.1)
            colsample_bytree = trial.suggest_discrete_uniform(
                'colsample_bytree', 0.5, 0.9, 0.1)

            xgboost_tuna = xgb.XGBClassifier(random_state=42,
                                             n_estimators=n_estimators,
                                             max_depth=max_depth,
                                             min_child_weight=min_child_weight,
                                             scale_pos_weight=scale_pos_weight,
                                             subsample=subsample,
                                             colsample_bytree=colsample_bytree,
                                             )
            xgboost_tuna.fit(X_train, y_train)
            tuna_pred_test = xgboost_tuna.predict(X_test)

            return ((accuracy_score(y_test, tuna_pred_test)))

        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=10, timeout=600)
        best_parameters = study.best_trial.params

        return best_parameters


class ElasticNetOptunaDriverClass:
    def set_tag(trial, df_new, features, label):
        def objective(trial):
            cat_col = df_new.select_dtypes(
                exclude=['int', 'float']).columns.values
            for c in cat_col:
                df_new[c] = df_new[c].astype('category')
                df_new[c] = lbl.fit_transform(df_new[c].astype(str))
            XRaw = df_new[features].values
            yRaw = df_new[label].values
            X_train, X_test, y_train, y_test = train_test_split(
                XRaw, yRaw, random_state=42)

            l1_ratio = trial.suggest_loguniform("l1_ratio", 0.1, 1)
            alpha = trial.suggest_loguniform("alpha", 0.1, 1)
            selection = trial.suggest_categorical(
                "selection", ['cyclic', 'random'])
            e_net = ElasticNet(
                alpha=alpha, l1_ratio=l1_ratio, selection=selection)
            e_net.fit(X_train, y_train)
            predictions = e_net.predict(X_test)
            loss = mean_absolute_error(predictions, y_test)
            return loss
        study = optuna.create_study(direction='minimize')
        study.optimize(objective, n_trials=20, timeout=600)
        best_parameters = study.best_trial.params

        return best_parameters


class PolynomialregressionOptunaDriverClass:
    def set_tag(trial, XRaw, yRaw):
        def objective(trial):
            X_train, X_test, y_train, y_test = train_test_split(
                XRaw, yRaw, random_state=42)
            degree = trial.suggest_int("degree", 2, 5)
            polynomial = PolynomialFeatures(degree=degree)
            x_train_transformed = polynomial.fit_transform(X_train)
            x_test_transformed = polynomial.fit_transform(X_test)
            clf = LinearRegression()
            clf.fit(x_train_transformed, y_train)
            predictions = clf.predict(x_test_transformed)
            loss = mean_absolute_error(predictions, y_test)
            return loss
        study = optuna.create_study(direction='minimize')
        study.optimize(objective, n_trials=5, timeout=600)
        best_parameters = study.best_trial.params

        return best_parameters


def randomforestclassifier(sparkDF, features, labels):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    rf_model = RandomForestClassifier()
    X_train, X_test, y_train, y_test = train_test_split(
        df[features], df[labels])
    rf_model.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % rf_model.score(X_test, y_test))

    data = {'model': rf_model,
            'X_test': X_test,
            'y_test': y_test,
            'label': labels,
            'columnNames': df.columns}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run bankdepositclfmanualmlHooks.ipynb
try:
	#sourcePreExecutionHook()

	bankdeposithistory = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/testdata/1715234309994_BankDepositHistory.csv', 'FilePath': '/Finance/Bank_deposit/BankDepositHistory.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'filename': 'BankDepositHistory.csv', 'viewFileName': 'BankDepositHistory.csv', 'is_header': 'Use Header Line', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'results_url': 'http://dnm.bfirst.ai:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run bankdepositclfmanualmlHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(bankdeposithistory,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "40.52", "stddev": "9.95", "min": "22", "max": "74", "missing": "0"}, "updatedLabel": "age"}, {"transformationsData": [{"feature_label": "job", "transformation_label": "String Indexer"}], "feature": "job", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "admin.", "max": "unknown", "missing": "0", "distinct": "12"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "job"}, {"transformationsData": [{"feature_label": "marital", "transformation_label": "String Indexer"}], "feature": "marital", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "divorced", "max": "single", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "marital"}, {"transformationsData": [{"feature_label": "education", "transformation_label": "String Indexer"}], "feature": "education", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "primary", "max": "unknown", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "education"}, {"transformationsData": [{"feature_label": "default", "transformation_label": "String Indexer"}], "feature": "default", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "no", "max": "yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "default"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "balance", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1186.08", "stddev": "1885.37", "min": "-1129", "max": "14481", "missing": "0"}, "updatedLabel": "balance"}, {"transformationsData": [{"feature_label": "housing", "transformation_label": "String Indexer"}], "feature": "housing", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "no", "max": "yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "housing"}, {"transformationsData": [{"feature_label": "loan", "transformation_label": "String Indexer"}], "feature": "loan", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "no", "max": "yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "loan"}, {"transformationsData": [{"feature_label": "contact", "transformation_label": "String Indexer"}], "feature": "contact", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "cellular", "max": "unknown", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "contact"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "day", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "15.83", "stddev": "8.36", "min": "2", "max": "31", "missing": "0"}, "updatedLabel": "day"}, {"transformationsData": [{"feature_label": "month", "transformation_label": "String Indexer"}], "feature": "month", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "apr", "max": "sep", "missing": "0", "distinct": "11"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "month"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "duration", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "715.8", "stddev": "496.17", "min": "8", "max": "3881", "missing": "0"}, "updatedLabel": "duration"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "campaign", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.64", "stddev": "2.38", "min": "1", "max": "19", "missing": "0"}, "updatedLabel": "campaign"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "pdays", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "5.94", "stddev": "41.03", "min": "-1", "max": "412", "missing": "0"}, "updatedLabel": "pdays"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "previous", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.14", "stddev": "1.33", "min": "0", "max": "27", "missing": "0"}, "updatedLabel": "previous"}, {"transformationsData": [{"feature_label": "poutcome", "transformation_label": "String Indexer"}], "feature": "poutcome", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "failure", "max": "unknown", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "poutcome"}, {"transformationsData": [{"feature_label": "deposit", "transformation_label": "String Indexer"}], "feature": "deposit", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "no", "max": "yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "deposit"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run bankdepositclfmanualmlHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=randomforestclassifier(autofe, ["age", "balance", "day", "duration", "campaign", "pdays", "previous", "job_stringindexer", "marital_stringindexer", "education_stringindexer", "default_stringindexer", "housing_stringindexer", "loan_stringindexer", "contact_stringindexer", "month_stringindexer", "poutcome_stringindexer"], "deposit_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()
